# Tarea 1
## Punto 1

Implemente las siguientes métricas de evaluación de IR usando python+numpy

In [3]:
import numpy as np

###  Precision (la relevancia es binaria)

In [4]:
def precision(q: list) -> float:
    """Calcula la precisión de una lista de resultados de búsqueda.

    Args:
        q (list): Una lista de resultados de búsqueda, donde cada elemento es un booleano que indica si el resultado es relevante o no.

    Raises:
        ValueError: Si 'q' no es una lista o si no contiene solo ceros (0) y unos (1).

    Returns:
        float: La precisión, que es la proporción de resultados relevantes entre todos los resultados devueltos.
    """
    if not isinstance(q, list):
        raise ValueError("q debe ser una lista")
    
    if all(item in [0, 1] for item in q):
        return sum(q) / len(q)
    else:
        raise ValueError("q debe contener solo ceros (0) y unos (1)")

In [5]:
relevance_query_1 = [False, False, False, True]
precision(relevance_query_1)

0.25

### Precision at K (la relevancia es binaria)

In [35]:
def precision_at_k(q: list, k: int) -> float:
    """Calcula la precisión de las k primeras posiciones de una lista de resultados de búsqueda.

    Args:
        q (list): Una lista de resultados de búsqueda, donde cada elemento es un booleano que indica si el resultado es relevante o no.
        k (int): El número de posiciones que se deben considerar.

    Raises:
        ValueError: Si "q" no es una lista o si no contiene solo ceros (0) y unos (1).
        ValueError: Si "k" no es un entero positivo.

    Returns:
        float: La precisión, que es la proporción de resultados relevantes entre las k primeras posiciones.
    """
    if not isinstance(q, list):
        raise ValueError("q debe ser una lista")
    
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k debe ser un entero positivo")
    
    if all(item in [0, 1] for item in q):
        return sum(q[:k]) / k
    else:
        raise ValueError("q debe contener solo ceros (0) y unos (1)")

In [40]:
relevance_query_1 = [False, False, False, True]
k = 1
precision_at_k(relevance_query_1, k)

0.0

### Recall at K (la relevancia es binaria)

In [20]:
def recall_at_k(q: list, k: int, number_relevant_docs: int) -> float:
    """Calcula la recall de las k primeras posiciones de una lista de resultados de búsqueda.

    Args:
        q (list): Una lista de resultados de búsqueda, donde cada elemento es un booleano que indica si el resultado es relevante o no.
        k (int): El número de posiciones que se deben considerar.
        number_relevant_docs (int): El número total de documentos relevantes.

    Raises:
        ValueError: Si 'q' no es una lista o si no contiene solo ceros (0) y unos (1).
        ValueError: Si 'k' no es un entero positivo.
        ValueError: Si 'number_relevant_docs' no es un entero no negativo.
    
    Returns (float):
        La recall, que es la proporción de documentos relevantes que se encuentran entre las k primeras posiciones.
    """
    if not isinstance(q, list):
        raise ValueError("q debe ser una lista")
    
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k debe ser un entero positivo")
    
    if not isinstance(number_relevant_docs, int) or number_relevant_docs < 0:
        raise ValueError("number_relevant_docs debe ser un entero no negativo")
    
    if all(item in [0, 1] for item in q):
        relevant_retrieved = sum(q[:k])
        return relevant_retrieved / number_relevant_docs
    else:
        raise ValueError("q debe contener solo ceros (0) y unos (1)")

In [41]:
relevance_query_1 = [False, False, False, True]
k = 1
number_relevant_docs = 4
recall_at_k(relevance_query_1, k, number_relevant_docs)

0.0

### Average precision (la relevancia es binaria)

In [27]:
def average_precision(q: list) -> float:
    """Calcula la media de la precisión en las k primeras posiciones para una lista de resultados de búsqueda.

    Args:
        q (list): Una lista de resultados de búsqueda, donde cada elemento es un booleano que indica si el resultado es relevante o no.

    Raises:
        ValueError: Si 'q' no es una lista o si no contiene solo ceros (0) y unos (1).

    Returns:
        float: La media de la precisión en las k primeras posiciones.
    """
    if not isinstance(q, list):
        raise ValueError("q debe ser una lista")
    
    if all(item in [0, 1] for item in q):
        total_relevant_docs = sum(q)
        accumulated_precision = 0.0
        relevant_count = 0
        
        if total_relevant_docs == 0:
            return 0.0

        for i, relevance in enumerate(q):
            if relevance == 1:
                relevant_count += 1
                accumulated_precision += precision_at_k(q, i+1)
        
        return accumulated_precision / total_relevant_docs
    else:
        raise ValueError("q debe contener solo ceros (0) y unos (1)")


In [30]:
relevance_query_2 = [False, True, False, True, True, True, True]
average_precision(relevance_query_2)

0.5961904761904762

### Mean average precision -MAP- (la relevancia es binaria)

In [31]:
def mean_average_precision(query_results: list) -> float:
    """Calcula la media de la precisión en las k primeras posiciones para un conjunto de resultados de búsqueda.

    Args:
        query_results (list): Un conjunto de resultados de búsqueda, donde cada elemento es una lista de booleanos que indican si el resultado es relevante o no.

    Raises:
        ValueError: Si 'query_results' no es una lista o si uno de sus elementos no es una lista.

    Returns:
        float: La media de la precisión en las k primeras posiciones.
    """
    total_queries = len(query_results)
    sum_average_precision = 0.0
    
    for q in query_results:
        sum_average_precision += average_precision(q)
    
    return sum_average_precision / total_queries

In [32]:
combined_query = [
    [False, False, False, True],
    [False, True, False, True, True, True, True]
]
mean_average_precision(combined_query)

0.4230952380952381

### DCG at K (la relevancia es un número natural)

$$
  DCG_K = R_1 + \sum_{i=2}^{K} \frac{R_i}{\log_2(i)}
$$

In [14]:
def dcg_at_k(relevance_vector: list, k: int) -> float:
    """Calcula el discounted cumulative gain (DCG) en las k primeras posiciones para una lista de valores de relevancia.

    Args:
        relevance_vector (list): Una lista de valores de relevancia, donde un valor de 1 indica que el documento es relevante y un valor de 0 indica que no es relevante.
        k (int): El número de posiciones que se deben considerar.

    Raises:
        ValueError: Si 'relevance_vector' no es una lista o si no contiene solo valores de 0 y 1.
        ValueError: Si 'k' no es un entero positivo.

    Returns:
        float: El discounted cumulative gain en las k primeras posiciones.
    """
    if not isinstance(relevance_vector, list):
        raise ValueError("relevance_vector debe ser una lista")
    
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k debe ser un entero positivo")
    
    dcg_value = relevance_vector[0]
    
    for i in range(1, min(k, len(relevance_vector))):
        dcg_value += relevance_vector[i] / (np.log2(i + 1))
    
    return dcg_value

In [15]:
relevance_query_3 = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
k = 6
dcg_at_k(relevance_query_3, k)

10.279642067948915

### NDCG at K (la relevancia es un número natural)

$$
NDCG_K = \frac{DCG_K}{IDCG_K}
$$

In [42]:
def ndcg_at_k(relevance_vector: list, k: int) -> float:
    """Calcula el normalized discounted cumulative gain (NDCG) en las k primeras posiciones para una lista de valores de relevancia.

    Args:
        relevance_vector (list): Una lista de valores de relevancia, donde un valor de 1 indica que el documento es relevante y un valor de 0 indica que no es relevante.
        k (int): El número de posiciones que se deben considerar.

    Raises:
        ValueError: Si 'relevance_vector' no es una lista o si no contiene solo valores de 0 y 1.
        ValueError: Si 'k' no es un entero positivo.

    Returns:
        float: El normalized discounted cumulative gain en las k primeras posiciones.
    """
    if not isinstance(relevance_vector, list):
        raise ValueError("relevance_vector debe ser una lista")
    
    if not isinstance(k, int) or k <= 0:
        raise ValueError("k debe ser un entero positivo")
    
    dcg_value = dcg_at_k(relevance_vector, k)
    ideal_dcg_value = dcg_at_k(sorted(relevance_vector, reverse=True), k)
    
    if ideal_dcg_value == 0:
        return 0.0
    
    ndcg_value = dcg_value / ideal_dcg_value
    return ndcg_value

In [43]:
relevance_query_3 = [4, 4, 3, 0, 0, 1, 3, 3, 3, 0]
k = 6
ndcg_at_k(relevance_query_3, k)

0.7424602308163405